In [ ]:
import cx_Oracle

In [ ]:
# Setup Environment
conn = cx_Oracle.connect(
    user="system",
    password="12341234",
    dsn="localhost/XEPDB1"
)
cursor = conn.cursor()

In [ ]:
# Create Tables
-- In SQL Developer or cx_Oracle cursor.execute() block
CREATE TABLE Banks (
    bank_id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    bank_name VARCHAR2(100) UNIQUE NOT NULL
);

CREATE TABLE Reviews (
    review_id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
    bank_id NUMBER,
    review_text CLOB,
    rating NUMBER,
    review_date DATE,
    sentiment_label VARCHAR2(20),
    sentiment_score FLOAT,
    topic_theme VARCHAR2(100),
    FOREIGN KEY (bank_id) REFERENCES Banks(bank_id)
);

In [ ]:
# Insert Data from df_copy
banks = df_copy['bank_name'].unique()
bank_ids = {}

for bank in banks:
    cursor.execute("INSERT INTO Banks (bank_name) VALUES (:1)", [bank])
    conn.commit()
    cursor.execute("SELECT bank_id FROM Banks WHERE bank_name = :1", [bank])
    bank_ids[bank] = cursor.fetchone()[0]

In [ ]:
for _, row in df_copy.iterrows():
    cursor.execute("""
        INSERT INTO Reviews (bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, topic_theme)
        VALUES (:1, :2, :3, TO_DATE(:4, 'YYYY-MM-DD'), :5, :6, :7)
    """, [
        bank_ids[row['bank_name']],
        row['review_text'],
        row['rating'],
        row['review_date'],
        row.get('sentiment_label', ''),
        float(row.get('sentiment_score', 0)),
        row.get('topic_theme', '')
    ])

conn.commit()

In [ ]:
# Verify Records
cursor.execute("SELECT COUNT(*) FROM Reviews")
print("Inserted Reviews:", cursor.fetchone()[0])